# Vectores, operaciones vectoriales básicas y Universal functions



Como cientificos de datos en algunos casos requeriremos crear nuestras propias funciones y es recomendable que se apeguen a las ufuncs, es decir que se puedan aplicar independiente de la dimensión del objeto al que se aplican, que soporten broadcasting y muy importante **que su definición evite ciclos y use operaciones vectorizadas y otras ufuncs** .

In [126]:
import numpy as np

## Ejercicio 1
En data science y programación científica es común necesitar vectores con valores dentro de cierto intervalo, pero con sub-intervalos internos con diference espaciamiento ,por ejemplo:

[0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ]
       
El primer sub-intervalo incrementa de 0.1 en 0.1 , el segundo de 0.01 en 0.01 , y el tercero de 0.1 en 0.1 nuevamente.

Usando unicamente NumPy crea un vector de este tipo con el nombre xs .

**nota**: no es permitido usar ciclos, debe realizarse con operaciones "vectorizadas" de numpy

In [127]:
## tu codigo aqui (~ 4 lineas de codigo)
x = np.linspace(0,0.5,6,True)
y = np.linspace(0.51,0.60,9,False)
z = np.linspace(0.6,1,5,True)
np.hstack((x,y,z))

array([0.  , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.7 , 0.8 , 0.9 , 1.  ])

## Ejercicio 2
Suavizado de curvas a través de promedios de n-puntos(medias móviles):

<img src="https://www.luisllamas.es/wp-content/uploads/2017/03/arduino-filtro-media-movil-ventana-5.png">

En el mundo la data comunmente posee "ruido" por lo cual muchas veces antes de hacer análisis o modelado se necesita "suavizar" las curvas para reducir este ruido, una técnica muy sencilla es  el suavizado por promedio de n-puntos(también llamado media móvil), esto significa que para cada punto Xn obtenemos una versión transformada(suavizada) que consiste en promediar n puntos cercanos a el, por ejemplo para n= 3 tenemos que:

$$XS_{n}  =  \frac{X_{n-1}+X_{n}+X_{n+1}}{3}$$

In [128]:
# las siguientes 3 lineas generan un conjunto de datos que se comportan segun una onda senoidal pero tienen
# ruido por lo cual vamos a suavizar usando promedio de 3 puntos
# en este ejercicio los generamos manualmente pero pensemos que son datos que pudieron ser generados con algún
# instrumento como un sensor, o bien estar almacenados en una base de datos
ruido = 0.1*np.random.randn(15) #el ruido comunmente se debe a aleatoriedad o captura no exacta de info.
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)

#xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()

for n in range(1,len(x)-1):
    xs[n] = (x[n-1] + x[n] + x[n+1])/3
    
print(x)
print(xs)

[ 0.01471794  0.41105121  0.70808007  0.98745325  0.98828971  0.77355553
  0.44343488 -0.10438194 -0.5471576  -0.67914463 -0.94954436 -0.93728116
 -0.84720712 -0.40034751 -0.09175244]
[ 0.01471794  0.37794974  0.70219484  0.89460767  0.91643283  0.73509338
  0.37086949 -0.06936822 -0.44356139 -0.72528219 -0.85532338 -0.91134421
 -0.7282786  -0.44643569 -0.09175244]


En este caso usamos un ciclo  para ejemplificar pero ya hemos mencionado que debemos evitarlos siempre que sea posible y usar operaciones "vectorizadas" ya que es mucho mas rápido al utilizar  código pre-compilado de C, en este ejercicio debemos remplazar el calculo de xs para hacerlo de manera "vectorizada" y eliminar el ciclo for. 

**tip**: usar slicing

In [129]:
import numpy as np
ruido = 0.1*np.random.randn(15) 
x = np.linspace(0,2*np.pi,15) + ruido
x = np.sin(x)

# xs es "x suavizado", inicialmente es una copia de x
xs =  x.copy()
# codigo aqui
x2 = xs[1:]
x2 = np.hstack((x2,xs[0]))
x3 = xs[2:]
x3 = np.hstack((x3,xs[0:2]))

print(x)
print(x2)
print(x3)
xs = print ((x+x2+x3)/3)

[ 0.16121464  0.35864857  0.79709969  0.96305939  0.96118427  0.86450576
  0.45157905  0.07046961 -0.45236714 -0.77362751 -0.96131258 -0.92799509
 -0.82657545 -0.39391826  0.08650751]
[ 0.35864857  0.79709969  0.96305939  0.96118427  0.86450576  0.45157905
  0.07046961 -0.45236714 -0.77362751 -0.96131258 -0.92799509 -0.82657545
 -0.39391826  0.08650751  0.16121464]
[ 0.79709969  0.96305939  0.96118427  0.86450576  0.45157905  0.07046961
 -0.45236714 -0.77362751 -0.96131258 -0.92799509 -0.82657545 -0.39391826
  0.08650751  0.16121464  0.35864857]
[ 0.43898763  0.70626922  0.90711445  0.92958314  0.75908969  0.46218481
  0.02322717 -0.38517501 -0.72910241 -0.88764506 -0.90529437 -0.71616293
 -0.3779954  -0.04873204  0.20212357]


## Ejercicio 3

NumPy posee logaritmos naturales , base 2 y base 10 , crear una *ufunc* que calcule el logaritmo en cualquier base n dada, la funcion debe recibir el tensor al cual calcular el logaritmo elemento por elemento y la base del logaritmo a calcular, por ejemplo:

`def logaritmo_n(x,n):`

In [130]:
## tu codigo aqui:
import numpy as np
def logaritmo_n(x,n):
    return (np.log(x)/np.log(n))
#Ingrese la base que le quiere aplicar al logaritmo
n = 2
#Ingrese los valores dentro del tensor que le quiere aplicar el logaritmo
x = np.array([0.25,0.75])
logaritmo_n(x,n)

array([-2.       , -0.4150375])

## Ejercicio 4
Para calcular la información de una distribución de probabilidad necesitamos calcular el producto entre la información de cada posible $x$ y su probabilidad, cada uno de estos elementos nos indica cuanta incerteza aporta cada $x$ a la entropía de la distribución. 

Para completar este ejercicio hay que crear una función que calcule lo descrito anteriormente, debe recibir un vector representando a la dist. de probabilidad y devoler otro vector con el termino de entropía para cada $x$.

Por ejemplo:

`calcular_entropia([0.25,0.75])`

Debe resultar en :

$[0.5    ,    0.31127812]$

**Nota**: la entropía es la suma sobre estos valores, en este caso no calculamos la entropía,unicamente los termimos de su sumatoria. La entropía es:

$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

In [131]:
## tu codigo aqui (~ 2 lineas de codigo)
def calcular_entropia(px):
    return np.nansum(-(px*np.log2(px)))
    
px = [0.25,0.75]
print(calcular_entropia(px))

0.8112781244591328


## Ejercicio 5

A partir de la información en un data warehouse se ha generado un modelo de datos, el valor de los clientes en el tiempo se ha determinado un modelo predictivo  no lineal que estima cuanta ganancia ha generado cierto cliente en 5 años en base a lo que ha gastado en su primer mes como cliente. 

El modelo tiene la forma:

$$g(x) = (x^{3} + 2x^{2} + e^{0.0001x} -\sqrt{2x})/50$$

Crear una función que calcule la ganancia para valores de gasto de nuevos clientes:

In [132]:
def g(x):
    ganancia = (np.power(x,3) + 2.0*(np.power(x,2.0))+np.exp(0.0001*x) - np.sqrt(2.0*x))/50
    return ganancia
    
gastos = np.array([100,50,25])
totalgastos = print(g(gastos))

[20399.73735829  2599.82010025   337.37862871]


Se ha determinado también que hay costos asociados a la comunicación y relación con los clientes(llamadas,mensajes electrónicos,etc) y que estos también tienen alta correlación con el monto que estos gastan en su primer més siguiendo el modelo:
$$c(x) = (x^{2} + x + log (0.0001x)-\sqrt{0.54x})/100$$

Cree una función para calcular este modelo sobre valores de de gasto de nuevos clientes , y luego cree una función  **ganancia_neta(x,g,c)** donde:
* x = vector conteniendo los valores de gasto del primer mes para nuevos clientes.
* g = función que estima la ganancia en función de x
* c = función que estima el costo en función de x

La función realiza el calculo simple **g(x) - c(x)** , ya que es posible que los modelos cambien en el tiempo es común recibir como parámetro el modelo o modelos hijos de un modelo mas grande , en este caso eso se logra recibiendo como parámetro las funciones de ganancia y costo.

In [133]:
def c(x):
    costos= (np.power(x,2)+ x + np.log(0.0001*x)-np.sqrt(0.54*x))/100
    return costos

def ganancia_neta(x, x2=g(x), x3=c(x)):
                  total = x2-x3
                  return(total)
        
#Ingrese valores en el tensor
x = np.array([100,50,25])
print(ganancia_neta(x))

[0.11518645 0.11466329]


# Funciones miscelaneas(agregación y estadistica descriptiva básica) 

Ya hemos trabajado algunas funciones de NumPy pero han sido funciones "elementwise" cuyo vector resultante tiene el mismo tamaño que el vector de entrada, existen muchas otras funciones que utilizamos en casos en los que necesitamos agregar,resumir cierto vector,incluyendo en este tipo de funciones algunas de estadistica descriptiva básica.

## Ejercicio 6
Usando argmax crear una función `mode(x)` para calcular la moda de un vector $x$, esta función debe devolver una tupla de 2 elementos de la forma:

(valor,conteo)

**tip** investigar y auxiliarse de otras funciones de numpy y np.argmax, no olvidar trabajar todo vectorizado, sin ciclos.

In [139]:
def mode(x):
    ## tu codigo aqui (~ 3 lineas de codigo):
    moda= np.bincount(x)
    return np.argmax(moda)

x= [4,4,4,4,5,5,2,2,2,2,2,22,2,7,7,7,8,8,8,8]
print(mode(x))

2


## Ejercicio 7
Crear una función para calcular la entropía  de una distribución de probabilidad representada como un vector.
$$S = -\sum_i^nP(x_{i})log_{2}P(x_{i})$$

**Nota**: a diferencia del ejercicio anterior donde solo calculamos los términos de la entropía, en este caso  si calculamos el valor completo de la entropía, que como mencionamos es una medida de incerteza en una distribución de probabilidad.

**recordatorio**: no usar ciclos solo operaciones vectorizadas.

In [135]:
def cross_entropy(Y,Yt):
    vector= Yt*np.log2(Y)
    suma = np.nansum(vector)
    return suma

y  = np.array([1.0,2.0,3.0])
yt = np.array([0.7,0.2,0.1])

cross_entropy(y,yt)

0.3584962500721156